In [1]:
import gzip
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy
import tensorflow as tf
from collections import defaultdict
from scipy.spatial import distance

In [7]:
from itertools import islice

with open("lthing_data/reviews.txt") as myfile:
    head = list(islice(myfile, 1000001))

In [8]:
for i in range(len(head)):
    exec(head[i])

In [9]:
count = 0
total_count = 0
for value in reviews.values():
    total_count+=1
#     print(value['flags'])
    if(len(value['flags']) != 0):
        count+=1

In [13]:
interactions = []
for item, user in reviews.keys():
    if('stars' in reviews[(item, user)].keys() and reviews[(item, user)]['stars'] >= 4.0 and len(reviews[(item, user)]['flags']) == 0):
        interactions.append((item, user, reviews[(item, user)]['unixtime'], reviews[(item, user)]['stars']))

In [15]:
random.shuffle(interactions)

In [16]:
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [17]:
friendsPerUser = defaultdict(set)
social_connections = open("lthing_data/edges.txt", 'r')
for l in social_connections:
    friend1, friend2 = l.strip().split(" ")
    if(friendsPerUser[friend1] == {}):
        friendsPerUser[friend1] = set(friend2)
    else:
        friendsPerUser[friend1].add(friend2)   
    if(friendsPerUser[friend2] == {}):
        friendsPerUser[friend2] = set(friend1)
    else:
        friendsPerUser[friend2].add(friend1)    

In [19]:
interactionsPerUser = defaultdict(list) #interactionsPerUser[u] = [t1, i1, r1] [t2,i2,r2]
userItems = defaultdict()
userIDs = {}
itemIDs = {}

for d in interactions:
    u,i,t,r = d[1], d[0], d[2],d[3]
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    #interactions.append((t,u,i))
    interactionsPerUser[u].append((t,i))

ITEMS = set(itemIDs.values())
userItems = defaultdict()
for u in interactionsPerUser:
    userItems[userIDs[u]] = set([itemIDs[x[1]] for x in interactionsPerUser[u]])

In [20]:
items = list(itemIDs.keys())

In [24]:
interactionsWithPrevious = []
for u in interactionsPerUser: interactionsPerUser[u].sort()

for u in interactionsPerUser:
#   print("good")
  neg_item = random.choice(list(ITEMS - userItems[userIDs[u]]))
  userInteractions = interactionsPerUser[u] #[t1,i1,r1][t2,i2,r2]
  if(len(userInteractions) == 0 or len(userInteractions) == 1 or len(userInteractions) == len(itemIDs)):continue
  for u_ind in range(1,len(userInteractions)):
    user, item, last_item, time = userIDs[u], itemIDs[userInteractions[u_ind][1]], itemIDs[userInteractions[u_ind-1][1]], userInteractions[u_ind][0]
    friendItems = defaultdict()
    for f in friendsPerUser[u]:
      if(f in interactionsPerUser.keys()):
#         print("yayy")
        friendInteractions = interactionsPerUser[f]#[t1,i1,r1][t2,i2,r2]
#         print(len(friendInteractions))
        for f_ind in range(len(friendInteractions)-1, 0, -1):
          if(friendInteractions[f_ind][0] < time):
#             print("yoo")
            friendItems[userIDs[f]] = itemIDs[friendInteractions[f_ind][1]]
            break
    interactionsWithPrevious.append([user, item, last_item, neg_item, friendItems])

#[u,i,l,n,[(f1,i1), f2:i2]]

In [27]:
K = 5
gamma_u = np.random.normal(0, 0.001, size=(len(userIDs),K))
gamma_i = np.random.normal(0, 0.001, size=(len(itemIDs),K))
theta_i = np.random.normal(0, 0.001, size=(len(itemIDs),K))
phi_i = np.random.normal(0, 0.001, size=(len(itemIDs),K))
psi_u = np.random.normal(0, 0.001, size=(len(userIDs),K))
b_i = np.random.normal(0, 0.001, size=(len(itemIDs)))
#NW = 2 * nUsers * K + 3 * nItems * K + nItems;

In [28]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [29]:
ALPHA = 1
biasReg = 0.00001
lamb = 0.00001

In [30]:
def prediction(u, i, l, friend_items):
  res = b_i[i] + np.dot(gamma_u[u],gamma_i[i]) + np.dot(theta_i[i],theta_i[l]);
  F = len(friend_items)**ALPHA
  for user_friend in friend_items:
      friend_item = friend_items[user_friend];
      res += (2/F) * sigmoid(np.dot(psi_u[u],psi_u[user_friend])) * np.dot(phi_i[i],phi_i[friend_item])
  return res

In [31]:
def updateFactors(user_id, pos_item_id, neg_item_id, last_item_id, friend_items, learn_rate):
    x_uti = prediction(user_id, pos_item_id, last_item_id, friend_items);
    x_utj = prediction(user_id, neg_item_id, last_item_id, friend_items);
    delta = 1 - sigmoid(x_uti - x_utj);
    b_i[pos_item_id] += learn_rate * (delta - biasReg * b_i[pos_item_id]);
    b_i[neg_item_id] += learn_rate * (-delta - biasReg * b_i[neg_item_id]);

    g_u_uk = gamma_u[user_id]
    g_i_ik = gamma_i[pos_item_id]
    g_i_jk = gamma_i[neg_item_id]
    gamma_u[user_id] += learn_rate * (delta * (g_i_ik - g_i_jk) - lamb * g_u_uk);
    gamma_i[pos_item_id] += learn_rate * (delta * g_u_uk - lamb * g_i_ik);
    gamma_i[neg_item_id] += learn_rate * (-delta * g_u_uk - lamb * g_i_jk); 

    t_i_ik = theta_i[pos_item_id]
    t_i_jk = theta_i[neg_item_id]
    t_l_lk = theta_i[last_item_id]
    theta_i[pos_item_id] += learn_rate * (delta * t_l_lk - lamb * t_i_ik);
    theta_i[neg_item_id] += learn_rate * (-delta * t_l_lk - lamb * t_i_jk);
    theta_i[last_item_id] += learn_rate * (delta * (t_i_ik - t_i_jk) - lamb * t_l_lk);

    friends_size = len(friend_items)
    F = friends_size
    
    psi_u_temp = psi_u[user_id]
    for user_friend_id in friend_items:
        friend_item_id = friend_items[user_friend_id]
        phi_i_f = np.dot(phi_i[pos_item_id],phi_i[friend_item_id])
        phi_j_f = np.dot(phi_i[neg_item_id],phi_i[friend_item_id])
        psi_u_f = sigmoid(np.dot(psi_u_temp,psi_u[user_friend_id]))

        phi_i_k = phi_i[pos_item_id]
        phi_j_k = phi_i[neg_item_id]
        phi_l_k = phi_i[friend_item_id]
        psi_u_k = psi_u_temp;
        psi_f_k = psi_u[user_friend_id]

        psi_u[user_id] += learn_rate * (delta * 2 * (psi_f_k * psi_u_f * (1.0 - psi_u_f)) * (phi_i_f - phi_j_f) / F - lamb * psi_u_k / friends_size)
        psi_u[user_friend_id] += learn_rate * (delta * 2 * psi_u_k * psi_u_f * (1.0 - psi_u_f) * (phi_i_f -phi_j_f) / F - lamb * psi_f_k)
        phi_i[pos_item_id] += learn_rate * (delta * 2 * psi_u_f * phi_l_k / F - lamb * phi_i_k / friends_size)
        phi_i[neg_item_id] += learn_rate * (-delta * 2 * psi_u_f * phi_l_k / F - lamb * phi_j_k / friends_size)
        phi_i[friend_item_id] += learn_rate * (delta * 2 * psi_u_f * (phi_i_k - phi_j_k) / F - lamb * phi_l_k)

In [33]:
def objective(interactionsWithPrevious):
  obj = []
  for user_id, pos_item_id, last_item_id, neg_item_id, friend_items in interactionsWithPrevious:
      x = prediction(user_id, pos_item_id, last_item_id, friend_items)
      x_ = prediction(user_id, neg_item_id,last_item_id, friend_items)
      obj.append(np.log(sigmoid(x-x_)))
  return -sum(obj)/len(obj)

In [ ]:
iterations = 10
learn_rate = 0.07
for i in range(0,iterations):
#   print(interactionsWithPrevious[:5])  
  for user, pos_item, last_item, neg_item, friend_items in interactionsWithPrevious:
    updateFactors(user, pos_item, neg_item, last_item, friend_items, learn_rate)
  #if i%5 == 0:
  obj = objective(interactionsWithPrevious)
  print(obj)

In [46]:
userAUC = defaultdict(int)
for u in interactionsPerUser:
  userAUC[userIDs[u]] = 0

In [47]:
def AUCu(user, pos_item, last_item, neg_item, friend_items):
  win = 0
  sp = prediction(user, pos_item, last_item, friend_items)
  #neg_item = random.choice(list(ITEMS - userItems[user]))
  sn = prediction(user, neg_item, last_item, friend_items)
  if sp >= sn:
      win += 1
  userAUC[user]+=win

In [48]:
def AUC():
  av = []
  for user, pos_item, last_item, neg_item, friend_items in interactionsWithPrevious:
      AUCu(user, pos_item, last_item, neg_item, friend_items)
  for u in interactionsPerUser.keys():
      if len(interactionsPerUser[u]) == 0: continue
      av.append(userAUC[userIDs[u]]/len(interactionsPerUser[u]))
  return sum(av) / len(av)

In [ ]:
print(AUC())